In [1]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import FunctionTool, ToolSet
from tools.user_functions import user_functions # user functions which can be found in a user_functions.py file.
from dotenv import load_dotenv

In [2]:
# Load environment variables from .env file
load_dotenv(override=True)

True

In [3]:
# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# It should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Customers need to login to Azure subscription via Azure CLI and set the environment variables

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

In [ ]:
# Initialize agent toolset with user functions
functions = FunctionTool(user_functions)
toolset = ToolSet()
toolset.add(functions)

agent = project_client.agents.create_agent(
    model=os.environ["O_MODEL_DEPLOYMENT_NAME"], 
    name="my-agent", 
    instructions="You are a weather bot. Use the provided functions to help answer questions.", 
    toolset=toolset
)
print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_jROJkw5rH3x4slhMokQCFMbb


In [5]:
# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="Hello, send an email with the datetime and weather information in New York?",
)
print(f"Created message, ID: {message.id}")

Created thread, ID: thread_YIn0WMbohyoE6Gs6ZBcTEJNY
Created message, ID: msg_CttsW6zhVAuPilZtGk0EmS7o


In [6]:
# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")
if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED


In [7]:
# Delete the assistant when done
project_client.agents.delete_agent(agent.id)
print("Deleted agent")


Deleted agent


In [8]:
# Fetch and log all messages
messages = project_client.agents.list_messages(thread_id=thread.id)
# Print all messages in the thread
for msg in reversed(messages.data):
    role = msg.role.capitalize()
    content = msg.content[0]['text']['value'] if msg.content else ''
    print(f"{role}: {content}\n")

User: Hello, send an email with the datetime and weather information in New York?

Assistant: The datetime must come from your end. Based on my current capabilities, here's the weather information for New York:

- Weather: Sunny
- Temperature: 25°C

